<a href="https://colab.research.google.com/github/Bhavin52/Case_Study/blob/main/SVM_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [93]:
salary_train = pd.read_csv("/content/SalaryData_Train.csv")
salary_test = pd.read_csv("/content/SalaryData_Test.csv")
string_columns=["workclass","education","maritalstatus","occupation","relationship","race","sex","native"]

In [94]:
salary_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [95]:
salary_test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [96]:
salary_train.shape

(1507, 14)

In [97]:
salary_test.shape

(752, 14)

In [98]:
salary_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507 entries, 0 to 1506
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            1507 non-null   int64 
 1   workclass      1507 non-null   object
 2   education      1507 non-null   object
 3   educationno    1507 non-null   int64 
 4   maritalstatus  1507 non-null   object
 5   occupation     1507 non-null   object
 6   relationship   1507 non-null   object
 7   race           1507 non-null   object
 8   sex            1507 non-null   object
 9   capitalgain    1507 non-null   int64 
 10  capitalloss    1507 non-null   int64 
 11  hoursperweek   1507 non-null   int64 
 12  native         1507 non-null   object
 13  Salary         1507 non-null   object
dtypes: int64(5), object(9)
memory usage: 165.0+ KB


In [99]:
salary_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            752 non-null    int64 
 1   workclass      752 non-null    object
 2   education      752 non-null    object
 3   educationno    752 non-null    int64 
 4   maritalstatus  752 non-null    object
 5   occupation     752 non-null    object
 6   relationship   752 non-null    object
 7   race           752 non-null    object
 8   sex            752 non-null    object
 9   capitalgain    752 non-null    int64 
 10  capitalloss    752 non-null    int64 
 11  hoursperweek   752 non-null    int64 
 12  native         752 non-null    object
 13  Salary         752 non-null    object
dtypes: int64(5), object(9)
memory usage: 82.4+ KB


In [100]:
salary_train.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1507.0,38.593895,12.823437,17.0,29.0,37.0,47.0,90.0
educationno,1507.0,10.193762,2.460771,1.0,9.0,10.0,13.0,16.0
capitalgain,1507.0,904.488388,6214.441668,0.0,0.0,0.0,0.0,99999.0
capitalloss,1507.0,98.604512,421.903250,0.0,0.0,0.0,0.0,2415.0
hoursperweek,1507.0,41.099536,11.686733,1.0,40.0,40.0,45.0,99.0


In [101]:
salary_test.describe().T

,count,mean,std,min,25%,50%,75%,max
age,752.0,37.474734,12.742535,17.0,27.0,36.0,46.0,80.0
educationno,752.0,10.090426,2.541232,1.0,9.0,10.0,13.0,16.0
capitalgain,752.0,1361.049202,8517.782930,0.0,0.0,0.0,0.0,99999.0
capitalloss,752.0,84.730053,389.834774,0.0,0.0,0.0,0.0,2444.0
hoursperweek,752.0,41.265957,11.738201,8.0,40.0,40.0,45.0,99.0


In [102]:
salary_train.corr()

,age,educationno,capitalgain,capitalloss,hoursperweek
age,1.000000,0.031113,0.096363,0.067368,0.111105
educationno,0.031113,1.000000,0.110773,0.062904,0.178525
capitalgain,0.096363,0.110773,1.000000,-0.034039,0.082916
capitalloss,0.067368,0.062904,-0.034039,1.000000,0.066839
hoursperweek,0.111105,0.178525,0.082916,0.066839,1.000000


In [103]:
salary_test.corr()

,age,educationno,capitalgain,capitalloss,hoursperweek
age,1.000000,-0.000670,0.093691,0.026186,0.142206
educationno,-0.000670,1.000000,0.120271,0.123655,0.106103
capitalgain,0.093691,0.120271,1.000000,-0.034776,0.117978
capitalloss,0.026186,0.123655,-0.034776,1.000000,0.055973
hoursperweek,0.142206,0.106103,0.117978,0.055973,1.000000


In [ ]:
sns.pairplot(salary_test)

Data Processing_with label encoder

In [109]:
from sklearn import preprocessing
nb = preprocessing.LabelEncoder()


In [110]:
salary_train["workclass"] = nb.fit_transform(salary_train["workclass"])
salary_train["education"] = nb.fit_transform(salary_train["education"])
salary_train["maritalstatus"] = nb.fit_transform(salary_train["maritalstatus"])
salary_train["occupation"] = nb.fit_transform(salary_train["occupation"])
salary_train["relationship"] = nb.fit_transform(salary_train["relationship"])
salary_train["race"] = nb.fit_transform(salary_train["race"])
salary_train["sex"] = nb.fit_transform(salary_train["sex"])
salary_train["native"] = nb.fit_transform(salary_train["native"])
salary_train["Salary"] = nb.fit_transform(salary_train["Salary"])

In [111]:
salary_test["workclass"] = nb.fit_transform(salary_test["workclass"])
salary_test["education"] = nb.fit_transform(salary_test["education"])
salary_test["maritalstatus"] = nb.fit_transform(salary_test["maritalstatus"])
salary_test["occupation"] = nb.fit_transform(salary_test["occupation"])
salary_test["relationship"] = nb.fit_transform(salary_test["relationship"])
salary_test["race"] = nb.fit_transform(salary_test["race"])
salary_test["sex"] = nb.fit_transform(salary_test["sex"])
salary_test["native"] = nb.fit_transform(salary_test["native"])
salary_test["Salary"] = nb.fit_transform(salary_test["Salary"])

In [118]:
x_train=salary_train.iloc[:,:-1]
x_train

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,39,5,9,13,4,0,1,4,1,2174,0,40,31
1,50,4,9,13,2,3,0,4,1,0,0,13,31
2,38,2,11,9,0,5,1,4,1,0,0,40,31
3,53,2,1,7,2,5,0,2,1,0,0,40,31
4,28,2,9,13,2,9,5,2,0,0,0,40,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,28,2,11,9,4,11,1,4,0,2202,0,48,31
1503,34,3,9,13,2,9,0,4,1,0,0,60,31
1504,54,2,11,9,2,6,0,4,1,5178,0,50,31
1505,52,2,11,9,0,7,2,2,0,0,0,38,31


In [119]:
y_train=salary_train.iloc[:,-1]
y_train

0       0
1       0
2       0
3       0
4       0
       ..
1502    0
1503    1
1504    1
1505    0
1506    1
Name: Salary, Length: 1507, dtype: int64

In [120]:
x_test=salary_test.iloc[:,:-1]
x_test

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,25,2,1,7,4,6,3,2,1,0,0,40,22
1,38,2,11,9,2,4,0,4,1,0,0,50,22
2,28,1,7,12,2,10,0,4,1,0,0,40,22
3,44,2,15,10,2,6,0,2,1,7688,0,40,22
4,34,2,0,6,4,7,1,4,1,0,0,30,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,46,2,14,15,4,9,1,4,1,0,0,90,22
748,31,2,8,11,4,12,1,4,1,0,0,40,22
749,21,1,15,10,2,10,3,2,1,0,0,40,22
750,61,2,11,9,2,2,0,4,1,0,0,40,22


In [121]:
y_test= salary_test.iloc[:,-1]
y_test

0      0
1      0
2      1
3      1
4      0
      ..
747    0
748    0
749    0
750    1
751    0
Name: Salary, Length: 752, dtype: int64

SVM Model

In [122]:
model = SVC()
model.fit(x_train, y_train)

SVC()

Predicting model

In [123]:
y_pred = model.predict(x_test)
y_pred

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [126]:
print(confusion_matrix(y_test, y_pred))

[[563   4]
 [151  34]]


In [127]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.99      0.88       567
           1       0.89      0.18      0.30       185

    accuracy                           0.79       752
   macro avg       0.84      0.59      0.59       752
weighted avg       0.81      0.79      0.74       752



for Model improvement using grid search

In [128]:
param_grid = {'C' : [1, 5, 10, 15, 20], 'gamma' : [1, 0.1, 0.01, 0.001, 0.0001], 'kernel' : ['rbf']}

In [129]:
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,  cv = 5)

In [ ]:
grid.fit(x_train, y_train)

In [131]:
grid.best_params_

{'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}

In [132]:
grid_pred = grid.predict(x_test)

Model Evaluation

In [ ]:
grid_pred

In [135]:
print(confusion_matrix(y_test, grid_pred))

[[565   2]
 [131  54]]


In [136]:
print(classification_report(y_test, grid_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       567
           1       0.96      0.29      0.45       185

    accuracy                           0.82       752
   macro avg       0.89      0.64      0.67       752
weighted avg       0.85      0.82      0.78       752



In [ ]:
#model accuracy increase from 79% to 82%